GOAL: ID the tumor sequencing data to run on the st jude cloud.  
Also, I have too many SAMPLE_INFO files from St Jude, want to make sure I use the right one and probably archive the others.



In [1]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path
from collections import OrderedDict

sys.path.append('../src')
Path("out").mkdir(parents=True, exist_ok=True)

import data_imports

from pathlib import Path
import os
import shutil

pd.set_option('display.max_columns', None)

# So many SAMPLE INFO files, and some directories without sample_info...
What does it all mean?

DNANexus  
├── PedPanCan  
│   ├── Data Requested 2022-08-29 # this is a superset of the project-specific sets from the same date, plus 7792 more files.    
│   │   └── SAMPLE_INFO.txt  
│   ├── Data Requested 2022-08-29 - Pediatric Cancer Genome Project  
│   │   └── SAMPLE_INFO.txt  
│   ├── Data Requested 2022-08-29 Real Time Clinical Genomics  
│   │   └── SAMPLE_INFO.txt  
│   ├── Data Requested 2022-09-02 # this is a subset of 829 missing 1240 files.  
│   │   └── SAMPLE_INFO.txt  
│   ├── Data Requested 2022-09-06 # considering only tumor WGS, this is equivalent to 829.  
│   │   └── SAMPLE_INFO.txt  
│   └── restricted  
└── PedPanCancer ecDNA  
    ├── Data Requested 2022-09-06  
    │   └── SAMPLE_INFO.txt # this is the same as the eponymous file in PedPanCan.  
    └── restricted  

1. "Data Requested 2022-08-29/SAMPLE_INFO.txt" (829) is a superset of all other SAMPLE_INFO files on St Jude.
2. How does 829 compare to our local SAMPLE_INFO files of unknown provenance?  
   'SAMPLE_INFO PedPanCancer ecDNA.txt', 'SAMPLE_INFO_2022-03-02.txt', and 'SAMPLE_INFO_batch_2022-03-03.txt' contain samples not in 829.

TODO: repeat this exercise excluding hematological malignancies.

In [ ]:
def read_sample_info(file):
    path=Path('/Users/ochapman/Library/CloudStorage/OneDrive-SanfordBurnhamPrebysMedicalDiscoveryInstitute/projects/2023-pedpancan/data/sjcloud',file)
    df = pd.read_csv(path,sep='\t')
    df['sample_type'] = df.sample_type.str.lower()
    df['file_name'] = df.file_path.map(lambda x: x.split('/')[-1])
    return df
def get_file_set(file):
    df = read_sample_info(file)
    st = ['diagnosis', 'relapse', 'metastasis', 'autopsy']
    df = df[(df.sequencing_type == 'WGS') & (df.file_type == 'BAM') & (df.sample_type.isin(st))  & df.file_path.str.endswith('.bam')]
    return set(df.file_name)
def text_venn2(s1, s2):
    print(f'size of set 1: {len(s1)}')
    print(f'size of set 2: {len(s2)}')
    print(f'Samples in s1 not in s2: {len(s1 - s2)}')
    print(f'Samples in s2 not in s1: {len(s2 - s1)}')
    print(f'Overlap: {len(s1 & s2)}')
def text_venn(file1, file2):
    s1 = get_file_set(file1)
    s2 = get_file_set(file2)
    text_venn2(s1,s2)

In [ ]:
DNANEXUS_ROOT = Path(Path.cwd(),'DNANexus')
si_ppc_829 = Path(DNANEXUS_ROOT,'PedPanCan','Data Requested 2022-08-29','SAMPLE_INFO.txt')
si_ppc_829_pcgp = Path(DNANEXUS_ROOT,'PedPanCan','Data Requested 2022-08-29 - Pediatric Cancer Genome Project','SAMPLE_INFO.txt')
si_ppc_829_rtcg = Path(DNANEXUS_ROOT,'PedPanCan','Data Requested 2022-08-29 Real Time Clinical Genomics','SAMPLE_INFO.txt')
si_ppc_92 = Path(DNANEXUS_ROOT,'PedPanCan','Data Requested 2022-09-02','SAMPLE_INFO.txt')
si_ppc_96 = Path(DNANEXUS_ROOT,'PedPanCan','Data Requested 2022-09-06','SAMPLE_INFO.txt')
si_ppce_96 = Path(DNANEXUS_ROOT,'PedPanCancer ecDNA','Data Requested 2022-09-06','SAMPLE_INFO.txt')

CLOUD_ROOT = Path('/Users/ochapman/Library/CloudStorage/OneDrive-SanfordBurnhamPrebysMedicalDiscoveryInstitute/projects/2023-pedpancan/data/sjcloud')
f1=Path(CLOUD_ROOT,'2022-09-08_SAMPLE_INFO.txt')
f2=Path(CLOUD_ROOT,'SAMPLE_INFO PedPanCan_clinical.txt')
f3=Path(CLOUD_ROOT,'SAMPLE_INFO PedPanCancer ecDNA.txt')
f4=Path(CLOUD_ROOT,'SAMPLE_INFO_2022-03-02.tsv')
f5=Path(CLOUD_ROOT,'SAMPLE_INFO_batch_2022-03-03.txt') # seems to be only WGS, mutually exclusive with f4.

In [ ]:
# These files are the same set
#text_venn(si_ppc_96, si_ppce_96)

# These are mutually exclusive
#text_venn(si_ppc_829_pcgp, si_ppc_829_rtcg)

# 829 includes 829pcgp, 829rtcg, and 7792 more files.
#text_venn2(get_file_set(si_ppc_829),get_file_set(si_ppc_829_pcgp) | get_file_set(si_ppc_829_rtcg))

# 829 is a superset of 92 + 1240 more files.
#text_venn(si_ppc_829, si_ppc_92)
#text_venn2(get_file_set(si_ppc_92),get_file_set(si_ppc_829_pcgp) | get_file_set(si_ppc_829_rtcg))

# 829 is a superset of 96 + 327 more files.
#text_venn(si_ppc_829,si_ppc_96)

text_venn(si_ppc_96,f1)

# 'SAMPLE_INFO PedPanCan_clinical.txt' is a subset of 829, missing 1639 files. Ok to delete.
#text_venn(si_ppc_829,f2)

# 'SAMPLE_INFO PedPanCancer ecDNA.txt' overlaps 829, but incompletely. 954 | 1494 | 45
text_venn(si_ppc_829,f3)

# 'SAMPLE_INFO_2022-03-02.txt' overlaps 829, but incompletely. 259 | 2189 | 245
text_venn(si_ppc_829,f4)

# 'SAMPLE_INFO_batch_2022-03-03.txt' mostly independent of 829. 1753 | 695 | 200
text_venn(si_ppc_829,f5)

# Inventory of the actual bam files we have in various locations

DNANexus  
├── PedPanCan  
│   ├── Data Requested 2022-08-29     
│   │   └── BAM - No WGS files of interest here.  
│   ├── Data Requested 2022-08-29 - Pediatric Cancer Genome Project  
│   │   └── BAM - this is empty.  
│   ├── Data Requested 2022-08-29 Real Time Clinical Genomics  
│   │   └── BAM - this is empty  
│   ├── Data Requested 2022-09-02  
│   │   └── BAM - this is empty  
│   ├── Data Requested 2022-09-06  
│   │   └── BAM - (almost) mutually exclusive with restricted.  
│   └── restricted  
│       └── BAM  
└── PedPanCancer ecDNA - These are copied from PedPanCan.  
    ├── Data Requested 2022-09-06  
    │   └── BAM  
    └── restricted  
        └── BAM  

Conclusion: only the directories copied from PedPanCan to PedPanCancer ecDNA have any WGS of interest here. 'Data Requested 2022-09-06' and 'restricted' directories are (almost) mutually exclusive.

In [ ]:
import dxpy
import os

In [ ]:
DX_AUTH_TOKEN = os.getenv('DX_AUTH_TOKEN')
dxpy.set_security_context({"auth_token_type": "Bearer", "auth_token": DX_AUTH_TOKEN})

In [ ]:
def validate_file(file):
    a = file.split('.')
    if a[-1] != 'bam':
        return False
    if a[1] != 'WholeGenome':
        return False
    b = a[0].split('_')
    t = b[1][0]
    if t == 'G' or t == 'X' or t == 'H' or t == 'I':
        return False
    return True
def get_dx_files(project_id,path,validate=True):
    gen = dxpy.find_data_objects(project=project_id, folder=path, recurse=False, describe=True)
    file_list = set()
    for file_obj in gen:
        file_name = file_obj['describe']['name']
        if validate:
            if not validate_file(file_name):
                continue
        file_list.add(file_name)
    return file_list

In [ ]:
ppc_id='project-Fk1P9x09ZgZJbb0q38XffZJB'
ppce_id='project-Fz9yBjQ9fj2QPbFY16B8JG8X'
ppc_829_path='/Data Requested 2022-08-29/BAM/'
ppc_96_path='/Data Requested 2022-09-06/BAM/'
res_path='/restricted/BAM/'

#ppc_829_files = get_dx_files(ppc_id,ppc_829_path)
#ppc_96_files = get_dx_files(ppc_id,ppc_96_path)
#ppc_res_files = get_dx_files(ppc_id,res_path)
ppce_res_files = get_dx_files(ppce_id,res_path)
ppce_96_files = get_dx_files(ppce_id,ppc_96_path)

In [ ]:
## comparing actual file sets

# ppce sets are almost mutually exclusive (overlap is {'SJBALL030248_D1.WholeGenome.bam', 'SJTALL030047_R1.WholeGenome.bam'})
#text_venn2(ppce_96_files,ppce_res_files)

# 96 file sets are the same again
#text_venn2(ppce_96_files,ppc_96_files)

# ppc_96 and restricted file sets are again almost mutually exclusive (overlap is the same as above)
#text_venn2(ppc_96_files,ppc_res_files)

# restricted file sets are the same again
#text_venn2(ppc_res_files,ppce_res_files)

In [ ]:
# comparing sample_info to actual file sets

# The sample_info for ppce_96 contains more samples than the actual BAM files
#text_venn2(get_file_set(si_ppce_96), ppce_96_files)

# The sample_info for 96 also contains info for most but not all of files in restricted. (45 files missing)
#text_venn2(get_file_set(si_ppce_96), ppce_res_files)

missing = ppce_res_files - get_file_set(si_ppce_96)

# f3 and f4 contain the 45 missing files. f4 has the same columns, so let's use that.
#text_venn2(missing,get_file_set(f4))

# should I use f3 or f4 to supplement info in f1?
f1s = set(read_sample_info(f1).columns)
f3s = set(read_sample_info(f3).columns)
f4s = set(read_sample_info(f4).columns)
f1s == f4s

missing_samples = list(map(lambda x: x.split('.')[0],missing))

In [ ]:
f1si = read_sample_info(si_ppce_96)
f4si = read_sample_info(f4)

d1_96 = f1si[f1si.file_name.isin(ppce_96_files) & f1si.file_name.map(validate_file)].copy()
d1_96['file_path'] = '/Data Requested 2022-09-06'+d1_96.file_path
d1_res = f1si[f1si.file_name.isin(ppce_res_files) & f1si.file_name.map(validate_file)].copy()
d1_res['file_path'] = '/restricted'+d1_res.file_path
d4 = f4si[f4si.file_name.isin(ppce_res_files) & f4si.file_name.map(validate_file) & ~(f4si.file_name.isin(d1.file_name))]

df = pd.concat([d1_res,d4,d1_96]).drop_duplicates('file_name')
print(len(df))
df.head()

df.to_csv('out/SAMPLE_INFO_SJ00.txt',sep='\t',index=False)

# Inventory AA results
We now have a list of all WGS files on PedPanCancer ecDNA of interest to us.
- Do we have AA results for all of these?
- Do we need to make another request for samples?

In [ ]:
def get_sj_cohort():
    asdf = data_imports.get_pedpancan_biosamples_from_AC()
    asdf = asdf[asdf.str.startswith('SJ')]
    return set(asdf)
def get_sample_info(file):
    df = pd.read_csv(file,sep='\t')
    # drop hematologic malignancies
    #df = df[~(df.attr_diagnosis_group == 'Hematologic Malignancy')]
    return df

In [ ]:
sj_set = get_sj_cohort()
all_dnanexus_wgs_set = set(get_sample_info('out/SAMPLE_INFO_SJ00.txt').sample_name)
all_dnanexus_wgs_set

text_venn2(sj_set,all_dnanexus_wgs_set)

# Old shit

In [ ]:
def get_sj_cohort():
    asdf = data_imports.get_pedpancan_biosamples_from_AC()
    asdf = asdf[asdf.str.startswith('SJ')]
    return set(asdf)
def read_sample_info(file):
    path=Path('/Users/ochapman/Library/CloudStorage/OneDrive-SanfordBurnhamPrebysMedicalDiscoveryInstitute/projects/2023-pedpancan/data/sjcloud',file)
    df = pd.read_csv(path,sep='\t')
    #TODO: remove hematologic malignancies.
    #return df[(df.sequencing_type == 'WGS') & (df.file_type == 'BAM') & (df.sample_type != 'germline')]
    #hm=[]
    return df[(df.sequencing_type == 'WGS') & (df.file_type == 'BAM') & (df.sample_type != 'Germline') & (~df.sj_diseases.isin(hm)) & df.file_path.str.endswith('.bam')]

In [ ]:
def text_venn(file):
    sj_cohort = get_sj_cohort()
    info = set(read_sample_info(file)['sample_name'])
    print(f'{file}')
    print(f'Samples in cohort not in SAMPLE_INFO: {len(sj_cohort - info)}')
    print(f'Samples in SAMPLE_INFO not in cohort: {len(info - sj_cohort)}')
    print(f'Overlap: {len(sj_cohort & info)}')
    

In [ ]:
text_venn(f1)
text_venn(f2)
text_venn(f3)
text_venn(f4)
text_venn(f5)

In [ ]:
read_sample_info(si_ppc_829).attr_diagnosis_group.unique()

In [ ]:
temp = read_sample_info(si_ppc_829)
temp[temp.attr_diagnosis_group == 'Not Available']